In [1]:
import sys
from pathlib import Path
import numpy as np
import polars as pl
import pickle
import logging

from IPython.display import Markdown, display

current_dir = Path.cwd()
if str(current_dir) not in sys.path:
    sys.path.insert(0, str(current_dir))

from src import (
    DataConfig, DataLoader, ModelingStrategy, ReleaseManager, BenchmarkPipeline, create_config
)
from datetime import date
from lets_plot import *
import optuna

LetsPlot.setup_html()
optuna.logging.set_verbosity(optuna.logging.CRITICAL)
np.random.seed(42)
print(current_dir)

from IPython.display import Markdown, display

/Users/ivn/Documents/PhD/Transformer Research/Code/Benchmarking/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/Users/ivn/Documents/PhD/Transformer Research/Code/Benchmarking


In [2]:
#get list of trainable SKUs
data_path = "/Users/ivn/Documents/PhD/Transformer Research/Code/Benchmarking/data/db_snapshot_offsite/train_data/processed/train_data_features.feather"
df_clean = pl.read_ipc(data_path)

sku_tuples_all = [(d['productID'], d['storeID']) for d in df_clean.select(pl.col("productID"), pl.col("storeID")).unique().to_dicts()]
print("total unseen skus: ", len(sku_tuples_all))

sku_exclude = (df_clean
 .group_by("storeID","productID")
 .agg(pl.col("date").first())
 .filter(pl.col("date") >= date(2016,1,1))
 .select("productID","storeID")
 )

sku_exclude = [(d['productID'], d['storeID']) for d in sku_exclude.select(pl.col("productID"), pl.col("storeID")).unique().to_dicts()]

sku_tuples_complete =  [sku for sku in sku_tuples_all if sku not in sku_exclude]
print("total unseen skus available for training: ", len(sku_tuples_complete))

total unseen skus:  7634
total unseen skus available for training:  7632


## Check SKUs for training data before 1.1.2016

In [144]:
(df_clean
.filter(
    (pl.col("storeID") == 1) &
    (pl.col("productID").is_in([2991, 2258]))
)
.sort("date")
.group_by(["storeID", "productID"])
.head(1)
.select("storeID", "productID", "date")
)

storeID,productID,date
i64,i64,date
1,2258,2016-02-13
1,2991,2016-01-16


In [137]:
(df_clean
 .group_by("storeID","productID")
 .agg(pl.col("date").first())
 .filter(pl.col("date") >= date(2016,1,1)))

storeID,productID,date
i64,i64,date
1,2991,2016-01-16
1,2258,2016-02-13


In [ ]:
split_date = date(2016,1,1)
train_summary=(df_clean
 .group_by("storeID", "productID")
 .agg([
     pl.col("date").min().alias("date_first"),
     pl.col("date").filter(pl.col("date") < split_date).count().alias("obs_train"),
     pl.col("date").filter(pl.col("date") >= split_date).count().alias("obs_test")
 ])
 .sort("storeID", "productID"))

In [139]:
train_summary.filter(pl.col("date_first") > split_date)

storeID,productID,date_first,obs_train,obs_test
i64,i64,date,u32,u32
1,2258,2016-02-13,0,100
1,2991,2016-01-16,0,128


In [142]:
train_summary.filter(pl.col("obs_train") < 150)

storeID,productID,date_first,obs_train,obs_test
i64,i64,date,u32,u32
1,664,2015-10-24,69,143
1,1020,2015-12-26,6,143
1,1098,2015-10-10,83,143
1,1669,2015-09-05,118,143
1,1790,2015-12-26,6,143
…,…,…,…,…
7,2130,2015-09-05,118,143
7,2677,2015-10-31,62,143
7,2692,2015-11-07,55,143


## Check pipeline with 3 SKUs and random HP

In [ ]:
data_config = DataConfig(
    mapping_path = 'data/feature_mapping_train.pkl',
    features_path = "/Users/ivn/Documents/PhD/Transformer Research/Code/Benchmarking/data/db_snapshot_offsite/train_data/processed/train_data_features.feather",
    target_path = "/Users/ivn/Documents/PhD/Transformer Research/Code/Benchmarking/data/db_snapshot_offsite/train_data/train_data/train_data_target.feather",
    split_date="2016-01-01",
)

sku_tuples=[(1912, 7), (377, 1), (715, 7)]

quantiles = [0.5, 0.7, 0.9, 0.95, 0.99]
pipeline = BenchmarkPipeline(data_config)

total unseen skus:  7634


In [9]:
results_lightning_std = pipeline.run_experiment(
    sku_tuples=sku_tuples,
    modeling_strategy=ModelingStrategy.INDIVIDUAL,
    model_type="xgboost_quantile",
    quantile_alphas=quantiles,
    hyperparameters = {
        "eta": 0.05,
        "max_depth": 8,
        "min_child_weight": 20,
        "subsample": 0.8,
        "colsample_bytree": 0.7,
        "gamma": 1.0,   
        "lambda": 10.0,
        "alpha": 1.0,
        "tree_method": "hist",
        "num_boost_round": 100,
        "seed": 42
},
    experiment_name="xgb_quantile_test",
    evaluate_on_test=True
)

Training models: 100%|██████████| 3/3 [00:01<00:00,  2.13it/s]


In [20]:
sample_result_1 = results_lightning_std.training_results[0]
print(f"Model type: {sample_result_1.model_type}")
print(f"Strategy: {sample_result_1.modeling_strategy.value}")
print(f"SKU tuples: {sample_result_1.sku_tuples}")
print(f"Quantile level: {sample_result_1.quantile_level}")
print(f"quantile_score: {sample_result_1.performance_metrics.get('quantile_score', 'N/A')}")
print(f"predictions: {sample_result_1.performance_metrics.get('predictions', 'N/A')}")
print("overall stats: ----------------------------------------------------")
print(f"number of models trained: {len(results_lightning_std.training_results)}")

Model type: xgboost_quantile
Strategy: individual
SKU tuples: [(1912, 7)]
Quantile level: 0.5
quantile_score: N/A
predictions: [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
overall stats: ----------------------------------------------------
number of models trained: 15


In [21]:
sample_result_1.performance_metrics

{'quantile_losses': array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 1. , 0. ,
        0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. ,
        0. , 0.5, 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 1. , 0. , 1. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0.5, 0. , 0. , 0. ,
        0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. ,
        0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ]),
 'predictions': array([-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0

In [9]:
release_manager = ReleaseManager()
output_dir = Path("./xgb_releases_2")
release_path = release_manager.create_complete_release(
    experiment_results=results_lightning_std,  # Your ExperimentResults from pipeline
    base_output_dir=output_dir
  )

# Check Data in pipeline and prototype for consistency

In [6]:
sku = (715, 7)
loader = DataLoader(data_config)
loader.load_data(lazy=False)
dataset = loader.prepare_modeling_dataset([sku], ModelingStrategy.INDIVIDUAL)

In [7]:
loader = DataLoader(data_config)
# Don't call load_data() at all - let prepare_modeling_dataset do it
dataset = loader.prepare_modeling_dataset([sku], ModelingStrategy.INDIVIDUAL)

In [8]:
# Access data
X_train_pipeline = dataset.X_train
y_train_pipeline = dataset.y_train
X_test_pipeline = dataset.X_test
y_test_pipeline = dataset.y_test

In [9]:
X_train_pipeline.shape

(741, 132)

In [34]:

# Data loading
mapping_path = "/Users/ivn/Documents/PhD/Transformer Research/Code/Benchmarking/data/feature_mapping_train.pkl"
features_path = "/Users/ivn/Documents/PhD/Transformer Research/Code/Benchmarking/data/db_snapshot_offsite/train_data/processed/train_data_features.feather"
target_path = "/Users/ivn/Documents/PhD/Transformer Research/Code/Benchmarking/data/db_snapshot_offsite/train_data/train_data/train_data_target.feather"

with open(mapping_path, 'rb') as f:
    mapping = pickle.load(f)

train_features = pl.read_ipc(features_path)
train_target = pl.read_ipc(target_path)

In [35]:
X = (train_features
     .lazy()
     .filter(pl.col('productID') == 715)
     .filter(pl.col('storeID') == 7)
     .drop_nulls()
     .sort("date","skuID")
     .collect())

X_bdIDs = (X
           .lazy()
           .select('bdID')
           .unique()
           .collect()
           .to_numpy()
           .flatten())

y = (train_target
     .lazy()
     .filter(pl.col("bdID").is_in(X_bdIDs))
     .join(
         X.lazy().select("bdID","skuID"), 
         on="bdID", 
         how="left")
     .sort("date","skuID")
     .collect())

In [36]:
split_date = pl.date(2016, 1, 1)
X_train = X.filter(pl.col("date") < split_date)
X_test = X.filter(pl.col("date") >= split_date)
y_train = y.filter(pl.col("date") < split_date)
y_test = y.filter(pl.col("date") >= split_date)

In [37]:
meta_cols = ['frequency',
 'idx',
 'bdID',
 'base_date',
 'date',
 'dateID',
 'skuID',
 'productID',
 'storeID',
 'companyID',
 'is_daily',
 'missing_value',
 'not_for_sale',
 'name',
 'name-2']

In [38]:
# Explicitly set dtype to float64 for all arrays
X_train = (X_train
            .sort("date","skuID")
            .select(pl.selectors.exclude(meta_cols)))

y_train = (y_train
            .sort("date","skuID")
            .select("target")
            .to_numpy())  # Also flatten to ensure 1D

X_test = (X_test
        .sort("date","skuID")
        .select(pl.selectors.exclude(meta_cols)))

y_test = (y_test
        .sort("date","skuID")
        .select("target"))

In [ ]:
#wrap them into print statements to see the output
print("Train set shape test:   ", X_train.shape == X_train_pipeline.shape )
print("Train label shape test: ", y_train.shape == y_train_pipeline.shape )
print(30*"-")
print("Test set shape test:  ", X_test.shape == X_test_pipeline.shape )
print("Test label shape test:", y_test.shape == y_test_pipeline.shape ) 

Train set shape test:    True
Train label shape test:  True
------------------------------
Test set shape test:   True
Test label shape test: True


In [ ]:
#test if prototype and pipeline data are the same
print("Training set check:    ", (X_train == X_train_pipeline).to_numpy().all())
print("Training label check:  ",(y_train == y_train_pipeline).all())
print(30*"-")
print("Test set check:   ", (X_test == X_test_pipeline).to_numpy().all())
print("Test label check: ",(y_test == y_test).to_numpy().all())  

Training set check:     True
Training label check:   True
------------------------------
Test set check:    True
Test label check:  True


# Test Hyperparameter mode

## Test Tune Run

In [3]:
data_path = "/Users/ivn/Documents/PhD/Transformer Research/Code/Benchmarking/data/db_snapshot_offsite/train_data/processed/train_data_features.feather"
df_clean = pl.read_ipc(data_path)

sku_tuples = [(d['productID'], d['storeID']) for d in df_clean.select(pl.col("productID"), pl.col("storeID")).unique().to_dicts()]
len(sku_tuples)

7634

In [4]:
data_config = DataConfig(
    mapping_path = 'data/feature_mapping_train.pkl',
    features_path = "/Users/ivn/Documents/PhD/Transformer Research/Code/Benchmarking/data/db_snapshot_offsite/train_data/processed/train_data_features.feather",
    target_path = "/Users/ivn/Documents/PhD/Transformer Research/Code/Benchmarking/data/db_snapshot_offsite/train_data/train_data/train_data_target.feather",
    split_date="2016-01-01",
)

pipeline = BenchmarkPipeline(data_config)

In [5]:

# Step 1: Tune hyperparameters
tune_result = pipeline.run_experiment(
    sku_tuples=sku_tuples,
    modeling_strategy=ModelingStrategy.COMBINED,
    model_type="xgboost_quantile",
    quantile_alphas=[0.5],
    mode="hp_tune",
    tune_on=10,
    tuning_config={'n_trials': 10, 'n_folds': 5}
)

Tuning dataset has 38247 training samples and 132 features.
Prepared training data with 38247 samples and 132 features.


In [15]:
tune_result.best_score

0.4355871870782814

In [16]:
tune_result.best_params

{'eta': 0.2251586017238223,
 'max_depth': 10,
 'min_child_weight': 26,
 'subsample': 0.9639446007829685,
 'colsample_bytree': 0.6507755877543546,
 'gamma': 0.6139963456942783,
 'reg_alpha': 6.768595449859619,
 'reg_lambda': 4.832153974114423,
 'n_estimators': 293}

## Run With tuned Params (done on server)

In [70]:
#Parameter from runs over 500 and over 1000 SKUs  each on 100 trials and 5 folds
hp_random = {
        "eta": 0.05,
        "max_depth": 8,
        "min_child_weight": 20,
        "subsample": 0.8,
        "colsample_bytree": 0.7,
        "gamma": 1.0,   
        "lambda": 10.0,
        "alpha": 1.0,
        "tree_method": "hist",
        "num_boost_round": 100,
        "seed": 42
}
hp_100 = {'eta': 0.299573707733717,
        'max_depth': 8,
        'min_child_weight': 19,
        'subsample': 0.696340395708422,
        'colsample_bytree': 0.6220507570163917,
        'gamma': 1.0789384910762259,
        'reg_alpha': 9.755231227570237,
        'reg_lambda': 8.295423481524228, 
        'n_estimators': 297}

hp_500 = {'eta': 0.2251586017238223,
        'max_depth': 10,
        'min_child_weight': 26,
        'subsample': 0.9639446007829685,
        'colsample_bytree': 0.6507755877543546,
        'gamma': 0.6139963456942783,
        'reg_alpha': 6.768595449859619,
        'reg_lambda': 4.832153974114423,
        'n_estimators': 293}

hp_1000 = {'eta': 0.2657057478526166, 
        'max_depth': 9,
        'min_child_weight': 19,
        'subsample': 0.6897467091557125, 
        'colsample_bytree': 0.9965497359024938,
        'gamma': 1.064228070424531, 
        'reg_alpha': 0.16585154768227728,
        'reg_lambda': 0.17000025072317992,
        'n_estimators': 297} 

In [71]:
hp_list = [hp_random,hp_100,hp_500,hp_1000]
quantiles = [0.5, 0.7, 0.9, 0.95, 0.99]

In [72]:
data_config = DataConfig(
    mapping_path = 'data/feature_mapping_train.pkl',
    features_path = "/Users/ivn/Documents/PhD/Transformer Research/Code/Benchmarking/data/db_snapshot_offsite/train_data/processed/train_data_features.feather",
    target_path = "/Users/ivn/Documents/PhD/Transformer Research/Code/Benchmarking/data/db_snapshot_offsite/train_data/train_data/train_data_target.feather",
    split_date="2016-01-01",
)

In [73]:
experiments= []
for i, hp in enumerate(hp_list, start=1):
    pipeline = BenchmarkPipeline(data_config)
    
    results = pipeline.run_experiment(
        sku_tuples= sku_tuples_complete[:100],
        modeling_strategy=ModelingStrategy.INDIVIDUAL,
        model_type="xgboost_quantile",
        quantile_alphas=quantiles,
        hyperparameters = hp,
        experiment_name=f"xgb_quantile_{i}",
        evaluate_on_test=True
    )
    release_manager = ReleaseManager()
    output_dir = Path("./xgb_releases_hp_full")
    release_path = release_manager.create_complete_release(
    experiment_results=results,  
    base_output_dir=output_dir
  )
    experiments.append([hp,results])
    print(f"Completed experiment {i}")


    

Training models: 100%|██████████| 100/100 [00:48<00:00,  2.05it/s]


Completed experiment 1


Training models: 100%|██████████| 100/100 [01:40<00:00,  1.00s/it]


Completed experiment 2


Training models: 100%|██████████| 100/100 [01:49<00:00,  1.09s/it]


Completed experiment 3


Training models: 100%|██████████| 100/100 [01:58<00:00,  1.19s/it]


Completed experiment 4


In [ ]:
#give results quick hp_name for later processing
hp_type = ["0","100","500","1000"]
for i,experiment in enumerate(experiments, start=0):
    experiment.append(hp_type[i])

#combine results into one dataframe for easier processing
results_dfs = []
for experiment in experiments:

    results_df = pl.DataFrame({
        "productID": [result.sku_tuples[0][0] for result in experiment[1].training_results],
        "storeID": [result.sku_tuples[0][1] for result in experiment[1].training_results],
        "quantile_level": [result.quantile_level for result in experiment[1].training_results],
        "mean_quantile_loss" : [result.performance_metrics.get('mean_quantile_loss') for result in experiment[1].training_results],
        "hp_type" : experiment[2]
    })
    results_dfs.append(results_df)


experiment_results = pl.concat(results_dfs, how="vertical")

#optinally save results as csv
#experiment_results.write_csv("my_experiment.csv", separator=",", include_header=True)


## Analyze and viz the runs on different HPs

### Analyse first

In [ ]:
#read int he experiment data from csv or use the one just created above
experiment_results = pl.read_csv("xgb_quantile_hp_tuning_1000.csv", separator=",") 
print("shape: ", experiment_results.shape) 
print(experiment_results.head())

shape:  (20000, 5)
shape: (5, 5)
┌───────────┬─────────┬────────────────┬────────────────────┬─────────┐
│ productID ┆ storeID ┆ quantile_level ┆ mean_quantile_loss ┆ hp_type │
│ ---       ┆ ---     ┆ ---            ┆ ---                ┆ ---     │
│ i64       ┆ i64     ┆ f64            ┆ f64                ┆ i64     │
╞═══════════╪═════════╪════════════════╪════════════════════╪═════════╡
│ 399       ┆ 1       ┆ 0.5            ┆ 0.185315           ┆ 0       │
│ 399       ┆ 1       ┆ 0.7            ┆ 0.237762           ┆ 0       │
│ 399       ┆ 1       ┆ 0.9            ┆ 0.111888           ┆ 0       │
│ 399       ┆ 1       ┆ 0.95           ┆ 0.08042            ┆ 0       │
│ 399       ┆ 1       ┆ 0.99           ┆ 0.055245           ┆ 0       │
└───────────┴─────────┴────────────────┴────────────────────┴─────────┘


In [132]:
def df_to_markdown(df):
    """Convert Polars DataFrame to markdown table"""
    # Get column names
    cols = df.columns
    
    # Create header
    header = "| " + " | ".join(cols) + " |"
    separator = "| " + " | ".join(["---"] * len(cols)) + " |"
    
    # Create rows
    rows = []
    for row in df.iter_rows():
        row_str = "| " + " | ".join(str(val) for val in row) + " |"
        rows.append(row_str)
    
    # Combine all parts
    markdown = "\n".join([header, separator] + rows)
    return markdown

In [133]:
df_agg = (experiment_results
 .group_by("hp_type","quantile_level")
 .agg(
    pl.col("mean_quantile_loss").mean().alias("avg"),
    pl.col("mean_quantile_loss").std().alias("std")
))

# Pivot the data
df = (df_agg
 .with_columns(
    (pl.col("avg").round(3).cast(str) + " ± " + pl.col("std").round(3).cast(str)).alias("value")
)
.pivot(
    index="hp_type",
    on="quantile_level",
    values="value"
)
.with_columns(
    pl.col("hp_type").cast(pl.Int32).alias("sort_order")
)
.sort("sort_order")
.drop("sort_order")
.select(["hp_type", "0.5", "0.7", "0.9", "0.95", "0.99"]))

# Find minimum avg value per quantile and add bold formatting
df_min = (df_agg
 .group_by("quantile_level")
 .agg(pl.col("avg").min().alias("min_avg")))

# Join and format with bold for minimum
df_formatted = (df_agg
 .join(df_min, on="quantile_level")
 .with_columns(
    pl.when(pl.col("avg") == pl.col("min_avg"))
    .then("**" + pl.col("avg").round(3).cast(str) + " ± " + pl.col("std").round(3).cast(str) + "**")
    .otherwise(pl.col("avg").round(3).cast(str) + " ± " + pl.col("std").round(3).cast(str))
    .alias("value")
)
.pivot(
    index="hp_type",
    on="quantile_level",
    values="value"
)
.with_columns(
    pl.col("hp_type").cast(pl.Int32).alias("sort_order")
)
.sort("sort_order")
.drop("sort_order")
.select(["hp_type", "0.5", "0.7", "0.9", "0.95", "0.99"]))

markdown_table = df_to_markdown(df_formatted)
display(Markdown(markdown_table))

| hp_type | 0.5 | 0.7 | 0.9 | 0.95 | 0.99 |
| --- | --- | --- | --- | --- | --- |
| 0 | 0.596 ± 1.433 | 0.715 ± 1.914 | 0.708 ± 2.357 | 0.681 ± 2.48 | 0.65 ± 2.58 |
| 100 | **0.478 ± 0.656** | **0.501 ± 0.627** | 0.324 ± 0.44 | 0.226 ± 0.361 | 0.108 ± 0.283 |
| 500 | 0.482 ± 0.721 | 0.511 ± 0.683 | 0.335 ± 0.521 | 0.236 ± 0.449 | 0.12 ± 0.38 |
| 1000 | 0.497 ± 0.642 | 0.528 ± 0.603 | **0.313 ± 0.425** | **0.214 ± 0.349** | **0.096 ± 0.284** |

### Viz 

In [ ]:
# Convert quantile_level to string for proper categorical handling
results_df_plot = experiment_results.with_columns(
    pl.col('quantile_level').cast(pl.Utf8).alias('quantile_level'),
    pl.col('hp_type').cast(str).alias('hp_type')
)

ggplot(results_df_plot, aes(x='quantile_level', y='mean_quantile_loss', fill='hp_type')) + \
    geom_boxplot(alpha=0.7, outlier_size=0.5, position='dodge') + \
    labs(
        title='Mean Quantile Loss Distribution by Quantile Level and HP Type',
        x='Quantile Level',
        y='Mean Quantile Loss',
        fill='HP Type'
    ) + \
    theme_minimal() + \
    theme(plot_title=element_text(size=14, face='bold')) + \
    ggsize(2000, 750)

In [ ]:
#Log scale with y-axis limits
results_df_plot = experiment_results.with_columns(
    pl.col('quantile_level').cast(pl.Utf8).alias('quantile_level'),
    pl.col('hp_type').cast(str).alias('hp_type')
)

ggplot(results_df_plot, aes(x='quantile_level', y='mean_quantile_loss', fill='hp_type')) + \
    geom_boxplot(alpha=0.7, outlier_size=0.5, position='dodge') + \
    scale_y_log10() + \
    coord_cartesian(ylim=[0.005, None]) + \
    labs(
        title='Mean Quantile Loss Distribution by Quantile Level and HP Type (Log Scale)',
        x='Quantile Level',
        y='Mean Quantile Loss (log scale)',
        fill='HP Type'
    ) + \
    theme_minimal() + \
    theme(plot_title=element_text(size=14, face='bold')) + \
    ggsize(1700, 800)

In [127]:
# Exclude multiple hp_types
results_df_plot = experiment_results.with_columns(
    pl.col('quantile_level').cast(pl.Utf8).alias('quantile_level'),
    pl.col('hp_type').cast(str).alias('hp_type')
).filter(
    pl.col('hp_type').is_in(["100", "500", "1000"])  # Only include these
)

ggplot(results_df_plot, aes(x='quantile_level', y='mean_quantile_loss', fill='hp_type')) + \
    geom_boxplot(alpha=0.7, outlier_size=0.5, position='dodge') + \
    scale_y_log10() + \
    labs(
        title='Mean Quantile Loss Distribution by Quantile Level and HP Type',
        x='Quantile Level',
        y='Mean Quantile Loss',
        fill='HP Type'
    ) + \
    theme_minimal() + \
    theme(plot_title=element_text(size=14, face='bold')) + \
    ggsize(2000, 750)

In [131]:
# Exclude multiple hp_types
results_df_plot = experiment_results.with_columns(
    pl.col('quantile_level').cast(pl.Utf8).alias('quantile_level'),
    pl.col('hp_type').cast(str).alias('hp_type')
).filter(
    pl.col('hp_type').is_in(["1000"])  # Only include these
)

ggplot(results_df_plot, aes(x='quantile_level', y='mean_quantile_loss')) + \
    geom_boxplot(alpha=0.7, outlier_size=0.5, position='dodge') + \
    scale_y_log10() + \
    labs(
        title='Mean Quantile Loss Distribution by Quantile Level for HP Type 1000',
        x='Quantile Level',
        y='Mean Quantile Loss',
        fill='HP Type'
    ) + \
    theme_minimal() + \
    theme(plot_title=element_text(size=14, face='bold')) + \
    ggsize(1700, 750)

In [143]:
#check the outliers:
results_df_plot

productID,storeID,quantile_level,mean_quantile_loss,hp_type
i64,i64,str,f64,str
399,1,"""0.5""",0.230769,"""1000"""
399,1,"""0.7""",0.248951,"""1000"""
399,1,"""0.9""",0.125175,"""1000"""
399,1,"""0.95""",0.09021,"""1000"""
399,1,"""0.99""",0.026294,"""1000"""
…,…,…,…,…
1598,7,"""0.5""",0.104895,"""1000"""
1598,7,"""0.7""",0.190909,"""1000"""
1598,7,"""0.9""",0.100699,"""1000"""
